### Assignment : Week 2
## Finding best policies in simple MDPs

Great work making the MDPs in Week 1!

In this assignment, we'll use the simplest RL techniques - Policy and Value iteration to find the best policies (which maximize the discounted total reward) in our MDPs from last week.

Feel free to use your own MDPs, or import them from the OpenAI Gym library.

You can start this assignment during/after reading Grokking Ch-3.

Let us recall the equation to find the value function of agent's states under a policy $\pi$ -
$$v_{\pi}(s) = \sum _{a} \pi(a|s) ~ \left( ~ \sum _{s', r} ~ p(s', r | s, a) ~ \left[r + \gamma v_{\pi}(s') \right] ~ \right)$$

We can observe that the value function $v_{\pi}$ has a lot of circular dependencies on different states. 

To solve such equations, one of the ways is to iteratively calculate the RHS and replace the LHS by it until the $v_{\pi}(s)$ values start to converge. 

The point of convergence makes all the equations simultaneously true and hence is the required solution.

Let us calculate the value functions for some policies in the MDPs we created last week.

## Environment 0 - Bandit Walk

Again, we consider the BW environment on Page 39.

Let's consider what seems to be the most natural policy - always go Right.

This environment is so simple, that we can simply calculate the value functions by hand.

Note that by convention for the terminal states, 
$$v_{\pi}(0) = v_{\pi}(2) = 0$$

Now, 
$$v_{\pi}(1) = 1 + \gamma \cdot v_{\pi}(2) = 1$$

Note both the summations just have one term due to the deterministic nature of the environment and the policy (check which summation was corresponding to which stochastic variable)

## Environment 1 - Slippery Walk

Let's now try to solve the SWF environment from Page 67 for the naturally adversarial policy - always go Left.

Since we have 5 coupled equations for states 1-5 with 5 unknown variables, we'll use Python to bruteforce the solution.

To align with Grokking, let us consider an unusual $\gamma = 1$.

In [36]:
# Step 0 is to import stuff

import gym, gym_walk
import numpy as np
from gym.envs.toy_text.frozen_lake import generate_random_map

In [37]:
# Step 1 is to get the MDP

env = gym.make('SlipperyWalkFive-v0')
swf_mdp = env.P
# swf_mdp

# Note that in Gym, action "Left" is "0" and "Right" is "1"

In [38]:
# Step 2 is to write the policy

pi = {
    0 : 0,
    1 : 0,
    2 : 0,
    3 : 0,
    4 : 0,
    5 : 0,
    6 : 0
}

# Or you can do it randomly
# pi = dict()
# for state in mdp:
#     pi[state] = np.random.choice(mdp[state].keys())

In [39]:
# Step 3 is computing the value function for this envi and policy

# Let us start with a random value function

val = dict()
for state in swf_mdp:
    val[state] = np.random.random()

# Since 0 and 6 are terminal states, we know their values are 0

val[0] = 0
val[6] = 0

#Or you could do it randomly, remember to set the terminal states to 0. You can also implement this while evaluating the value function using 
# val = dict()
# for state in mdp:
#     val[state] = np.random.random()
#     if mdp[state][0][0][0] == 0: # if the first action in the first outcome of the first state is 0, then it is a terminal state
#         val[state] = 0

#instead of doing thsi you can simply intialize the value function to 0 for all states 
# for state in swf_mdp:
#   val[state] = 0

In [40]:
def get_new_value_fn(val, mdp, pi, gamma = 1.0):
    
    new_val = dict()
    # Complete this function to get the new value function given the old value function and the policy
    for state in mdp:
        new_val[state]=0
        for prob,next_s,reward,done in mdp[state][pi[state]]:
            new_val[state]+=prob*(reward+gamma*val[next_s]*(not done))
    return new_val

In [41]:
#Use to above function to get the new value function, also print how many iterations it took to converge
def policy_evaluation(val, mdp, pi, epsilon=1e-10, gamma=1.0):
    count = 0
    # Complete this function to iteratively caluculate the value function until the difference between the new and old value function is less than epsilon
    # Also return the number of iterations it took to converge
    while(True):
        new_val=get_new_value_fn(val,mdp,pi,gamma=1.0)
        delta=max(abs(new_val[s]-val[s]) for s in val)
        if delta<epsilon:
            break
        val=new_val.copy()
        count+=1.0
    return val, count 

In [42]:
# Perform policy improvement using the polivy and the value function and return a new policy, the action value function should be a nested dictionary
def policy_improvement(val, mdp, gamma=1.0):
    new_pi = dict()
    q = dict()
    # Complete this function to get the new policy given the value function and the mdp
    for state in mdp:
        q[state]=dict()
        for action in mdp[state]:
            q[state][action]=0
            for prob,next_s,reward,done in mdp[state][action]:
                q[state][action]+=prob*(reward+gamma*val[next_s]*(not done))
            new_pi[state]=max(q[state],key=q[state].get)
    return new_pi, q


In [43]:
# Use the above functions to get the optimal policy and optimal value function and return the total number of iterations it took to converge
# Create a random policy and value function to start with or use the ones defined above
def policy_iteration(mdp, epsilon=1e-10, gamma=1.0):
    pi = dict()
    val = dict()
    count = 0
    # Complete this function to get the optimal policy and value function and return the total number of iterations it took to converge
    for state in mdp:
        pi[state]=np.random.choice(list(mdp[state].keys()))
        val[state]=0
    while(True):
        val,x=policy_evaluation(val,mdp,pi, epsilon=1e-10,gamma=1.0)
        new_pi,q=policy_improvement(val,mdp,gamma=1.0)
        if new_pi==pi:
            break
        count+=x
        pi=new_pi.copy()
    return pi, val, count

In [44]:
#Now perform value iteration, note that the value function is a dictionary and not a list, also return the number of iterations it took to converge
def value_iteration(mdp, gamma=1.0, epsilon=1e-10):
    val = {s: 0 for s in mdp}
    count = 0
    q = dict()
    # Complete this function to get the optimal policy, optimal value function and return the total number of iterations it took to converge
    while(True):
        count+=1
        for state in mdp:
            q[state]=dict()
            for action in mdp[state]:
                q[state][action]=0
                for prob,next_s,reward,done in mdp[state][action]:
                    q[state][action]+=prob*(reward+gamma*val[next_s]*(not done))
        if max(abs(val[state]-max(q[state].values())) for state in mdp)<epsilon:
            break
        val={state: max(q[state].values()) for state in mdp}
    pi = {s: max(q[s], key=q[s].get) for s in mdp}
    return pi, val, count
    

## Enviroment 2 - Frozen Lake

Repeat the above steps for the frozen lake environment. Don't create new functions , use the old functions.

You can also write a function `test_policy()` to test your policy after training to find the number of times you reached the goal state

In [45]:
env2 = gym.make('FrozenLake-v1',desc=generate_random_map(size=4))
mdp2 = env2.P

In [46]:
pi1, val1, count1 = policy_iteration(mdp2)
pi2, val2, count2 = value_iteration(mdp2)

In [47]:
def test_policy(pi, env, goalstate):
    # Complete this function to test the policy
    total_episodes=10000
    success=0
    for episode in range(total_episodes):
        state=env.reset()[0]
        done=False
        while(not done):
            action=pi[state]
            next_s, reward, done, trunc,dict=env.step(action)
            if(next_s==goalstate):
                success+=1
            state=next_s
    return (success/total_episodes)*100

final_pi, final_val, final_count=value_iteration(mdp2,gamma=1.0,epsilon=1e-10)
print(test_policy(final_pi,env2,15))

c:\Users\aksgu\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


100.0
